# Check if pdbdb data has changed 

Search PBDB to see of taxa lineage has changed

In [126]:
import sys
sys.path.append('../../../')

import pandas as pd
import numpy as np
import requests
import time

from config import CLEAN_DATA_DIR, OUTPUT_DIR, RAW_DATA_DIR

import scripts.normalize_taxa as nt
from scripts.normalize_data import remove_whitespace
import scripts.pbdb as pbdb

from scripts.pbdb import (
    get_parent_taxa, 
    PBDB_TAXA_NAME, 
    PBDB_TAXA_ID,
    check_multiple_pbdb_id
)
from scripts.shared_utils import (
    log_df
)


In [51]:
date = '2022-11-15'
old_taxa_list_file = OUTPUT_DIR/'taxa'/'LIMS'/f"taxa_list_{date}.csv"

date = '2023-02-08'
species_file = OUTPUT_DIR/'taxa'/'draft'/'LIMS'/'species'/f'species_list_{date}.csv'
genus_file = OUTPUT_DIR/'taxa'/'LIMS'/f'genera_pbdb_{date}.csv'
genus_higher_file = OUTPUT_DIR/'taxa'/'LIMS'/f'genera_and_higher_pbdb_{date}.csv'




In [50]:
def fetch_pbdb_data(url, df, rank):
    response = requests.get(url)
    if response.status_code == 200:
        data = response.json()["records"]
        if len(data) == 1:
            df.at[index, f'pbdb_taxon_id'] = str(data[0]["taxon_no"])
            df.at[index, f'pbdb_taxon_name'] = data[0]["taxon_name"]
            df.at[index, f'pbdb_taxon_rank'] = data[0]["taxon_rank"]

            df.at[index, f'{rank}_taxon_id'] = str(data[0]["taxon_no"])
            df.at[index, f'{rank}_taxon_name'] = data[0]["taxon_name"]

            round = 0
            get_parent_taxa(df, data[0]["parent_no"], data[0]["taxon_rank"], round, index, None)


    df.at[index, f'checked'] = True

# create species list with pbdb data

### create basic species files

In [106]:
taxa_df = pd.read_csv(old_taxa_list_file, dtype=str)
log_df(taxa_df)
# 4694

(4694, 29)


,Any taxon above genus,genus modifier,genus name,subgenera modifier,subgenera name,species modifier,species name,subspecies modifier,subspecies name,non-taxa descriptor,...,family_taxon_id,family_taxon_name,order_taxon_id,order_taxon_name,class_taxon_id,class_taxon_name,phylum_taxon_id,phylum_taxon_name,kingdom_taxon_id,kingdom_taxon_name
0,NaN,NaN,Euuvigerina,NaN,NaN,NaN,miozea,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,288974,Foraminifera,212476,Rhizaria
1,NaN,NaN,Euuvigerina,NaN,NaN,NaN,rodleyi,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,288974,Foraminifera,212476,Rhizaria
2,Foraminifera indet.,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,288974,Foraminifera,212476,Rhizaria
3,Pleurostomellidae indet.,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,155922,Pleurostomellidae,NaN,NaN,NaN,NaN,288974,Foraminifera,212476,Rhizaria
4,Ostracoda indet.,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,22826,Ostracoda,18891,Arthropoda,325038,Animalia


In [107]:
filter_taxa_df = taxa_df[taxa_df['Any taxon above genus'].isna()]
filter_taxa_df = filter_taxa_df[filter_taxa_df['species name'].notna()]
log_df(filter_taxa_df)

filter_taxa_df.loc[~filter_taxa_df['species name'].str.contains('spp\.|sp\..*?', regex=True), 'genus species name'] = filter_taxa_df['genus name'] + ' ' + filter_taxa_df['species name']

filter_taxa_df['genus species name'] = filter_taxa_df['genus species name'].str.strip()

log_df(filter_taxa_df)
# 4587

(4587, 29)
(4587, 30)


,Any taxon above genus,genus modifier,genus name,subgenera modifier,subgenera name,species modifier,species name,subspecies modifier,subspecies name,non-taxa descriptor,...,family_taxon_name,order_taxon_id,order_taxon_name,class_taxon_id,class_taxon_name,phylum_taxon_id,phylum_taxon_name,kingdom_taxon_id,kingdom_taxon_name,genus species name
0,NaN,NaN,Euuvigerina,NaN,NaN,NaN,miozea,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,288974,Foraminifera,212476,Rhizaria,Euuvigerina miozea
1,NaN,NaN,Euuvigerina,NaN,NaN,NaN,rodleyi,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,288974,Foraminifera,212476,Rhizaria,Euuvigerina rodleyi
6,NaN,NaN,Nodosaria,NaN,NaN,NaN,spp.,NaN,NaN,NaN,...,Nodosariidae,429322,Nodosariida,428875,Nodosariata,288974,Foraminifera,212476,Rhizaria,NaN
7,NaN,NaN,Cibicides,NaN,NaN,NaN,spp.,NaN,NaN,NaN,...,Cibicididae,NaN,NaN,NaN,NaN,288974,Foraminifera,212476,Rhizaria,NaN
8,NaN,NaN,Brizalina,NaN,NaN,NaN,spp.,NaN,NaN,NaN,...,Bolivinidae,NaN,NaN,NaN,NaN,288974,Foraminifera,212476,Rhizaria,NaN


In [108]:
species_df = filter_taxa_df[['genus species name']].copy().dropna().drop_duplicates().reset_index(drop=True)

species_df['pbdb_taxon_id'] = pd.NA
species_df['checked'] = False

log_df(species_df)

species_df.to_csv(species_file, index=False)
# 3267

(3267, 3)


### add pbdb info

In [109]:
species_df = pd.read_csv(species_file)
log_df(species_df)

(3267, 3)


,genus species name,pbdb_taxon_id,checked
0,Euuvigerina miozea,NaN,False
1,Euuvigerina rodleyi,NaN,False
2,Candeina nitida,NaN,False
3,Dentoglobigerina altispira,NaN,False
4,Dentoglobigerina baroemoenensis,NaN,False


In [117]:
for index, row in species_df.iterrows():         
    if index % 50 == 0:
        print(index, end=' ')    

    
    if row['genus species name'] == 'Hirsutella hirsuta':
        row['genus species name'] = 'Globorotalia (Hirsutella) hirsuta'
        
    if row['checked'] == 'True' or row['checked'] == True:
        continue
        
    time.sleep(0.25)
        
    url =  PBDB_TAXA_NAME +  row['genus species name']
    fetch_pbdb_data(url, species_df, 'species')


0 50 100 150 200 250 300 350 400 450 500 550 600 650 700 750 800 850 900 950 1000 1050 1100 1150 1200 1250 1300 1350 1400 1450 1500 1550 1600 1650 1700 1750 1800 1850 1900 1950 2000 2050 2100 2150 2200 2250 2300 2350 2400 2450 2500 2550 2600 2650 2700 2750 2800 2850 2900 2950 3000 3050 3100 3150 3200 3250 

In [118]:
species_df[species_df['checked'] == True].shape

(3267, 21)

In [119]:
log_df(species_df)

(3267, 21)


,genus species name,pbdb_taxon_id,checked,pbdb_taxon_name,pbdb_taxon_rank,species_taxon_id,species_taxon_name,genus_taxon_id,genus_taxon_name,family_taxon_id,...,phylum_taxon_id,phylum_taxon_name,kingdom_taxon_id,kingdom_taxon_name,order_taxon_id,order_taxon_name,class_taxon_id,class_taxon_name,unranked clade_taxon_id,unranked clade_taxon_name
0,Euuvigerina miozea,NaN,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Euuvigerina rodleyi,NaN,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Candeina nitida,422278,True,Candeina nitida,species,422278,Candeina nitida,1053,Candeina,422277,...,288974,Foraminifera,212476,Rhizaria,NaN,NaN,NaN,NaN,NaN,NaN
3,Dentoglobigerina altispira,402661,True,Dentoglobigerina altispira,species,402661,Dentoglobigerina altispira,1264,Dentoglobigerina,82191,...,288974,Foraminifera,212476,Rhizaria,NaN,NaN,NaN,NaN,NaN,NaN
4,Dentoglobigerina baroemoenensis,422289,True,Dentoglobigerina baroemoenensis,species,422289,Dentoglobigerina baroemoenensis,1264,Dentoglobigerina,82191,...,288974,Foraminifera,212476,Rhizaria,NaN,NaN,NaN,NaN,NaN,NaN


In [120]:
species_df.to_csv(species_file, index=False)

# create genera and higher file with pbdb data

### create basic genera and higher file

In [42]:
taxa_df = pd.read_csv(old_taxa_list_file, dtype=str)
log_df(taxa_df)
# 4694

(4694, 29)


,Any taxon above genus,genus modifier,genus name,subgenera modifier,subgenera name,species modifier,species name,subspecies modifier,subspecies name,non-taxa descriptor,...,family_taxon_id,family_taxon_name,order_taxon_id,order_taxon_name,class_taxon_id,class_taxon_name,phylum_taxon_id,phylum_taxon_name,kingdom_taxon_id,kingdom_taxon_name
0,NaN,NaN,Euuvigerina,NaN,NaN,NaN,miozea,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,288974,Foraminifera,212476,Rhizaria
1,NaN,NaN,Euuvigerina,NaN,NaN,NaN,rodleyi,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,288974,Foraminifera,212476,Rhizaria
2,Foraminifera indet.,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,288974,Foraminifera,212476,Rhizaria
3,Pleurostomellidae indet.,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,155922,Pleurostomellidae,NaN,NaN,NaN,NaN,288974,Foraminifera,212476,Rhizaria
4,Ostracoda indet.,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,22826,Ostracoda,18891,Arthropoda,325038,Animalia


In [43]:
set(taxa_df['pbdb_taxon_rank'])

{'class',
 'family',
 'genus',
 nan,
 'order',
 'phylum',
 'species',
 'subclass',
 'subgenus',
 'suborder',
 'subphylum',
 'superclass',
 'superfamily',
 'unranked clade'}

set pbdb fields for species records to genus

In [44]:
taxa_df.loc[taxa_df['pbdb_taxon_rank'] == 'species', 'pbdb_taxon_name'] = taxa_df['genus_taxon_name']
taxa_df.loc[taxa_df['pbdb_taxon_rank'] == 'species', 'pbdb_taxon_id'] = taxa_df['genus_taxon_id']
taxa_df.loc[taxa_df['pbdb_taxon_rank'] == 'species', 'pbdb_taxon_rank'] = 'genus'

In [45]:
# taxa_df = taxa_df[taxa_df['Any taxon above genus'].isna()]
# log_df(taxa_df)

# 4588

In [46]:
set(taxa_df['pbdb_taxon_rank'])

{'class',
 'family',
 'genus',
 nan,
 'order',
 'phylum',
 'subclass',
 'subgenus',
 'suborder',
 'subphylum',
 'superclass',
 'superfamily',
 'unranked clade'}

In [47]:
cols = ['genus_taxon_name', 'pbdb_taxon_rank','pbdb_taxon_name', 'pbdb_taxon_id']
genus_df = taxa_df[cols].copy().reset_index(drop=True)
genus_df = genus_df.dropna( how='all')


# genus_df = genus_df.drop_duplicates(subset=['pbdb_taxon_name']) # 1063
# genus_df = genus_df.drop_duplicates(subset=['genus_taxon_name']) #1060
genus_df = genus_df.drop_duplicates() #1063

genus_df['checked'] = False

log_df(genus_df)

# 1063 genus and subgenera
# 1120 higher and genus

(1120, 5)


,genus_taxon_name,pbdb_taxon_rank,pbdb_taxon_name,pbdb_taxon_id,checked
0,Euuvigerina,genus,Euuvigerina,1408,False
2,NaN,phylum,Foraminifera,288974,False
3,NaN,family,Pleurostomellidae,155922,False
4,NaN,class,Ostracoda,22826,False
5,NaN,subclass,Textulariia,434530,False


In [48]:
set(genus_df['pbdb_taxon_rank'])

{'class',
 'family',
 'genus',
 'order',
 'phylum',
 'subclass',
 'subgenus',
 'suborder',
 'subphylum',
 'superclass',
 'superfamily',
 'unranked clade'}

In [52]:
genus_df.to_csv(str(genus_higher_file), index=False)

### add pbdb info

In [61]:
genus_df = pd.read_csv(genus_higher_file, dtype=str)
log_df(genus_df)

# 1063 genus and subgenera
# 1120 genus and higher

(1120, 18)


,genus_taxon_name,pbdb_taxon_rank,pbdb_taxon_name,pbdb_taxon_id,checked,genus_taxon_id,phylum_taxon_id,phylum_taxon_name,kingdom_taxon_id,kingdom_taxon_name,family_taxon_id,family_taxon_name,order_taxon_id,order_taxon_name,class_taxon_id,class_taxon_name,unranked clade_taxon_id,unranked clade_taxon_name
0,Euuvigerina,genus,Euuvigerina,1408,True,1408,288974,Foraminifera,212476,Rhizaria,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,phylum,Foraminifera,288974,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,family,Pleurostomellidae,155922,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,class,Ostracoda,22826,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,subclass,Textulariia,434530,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [62]:
genus_df[genus_df['checked'] != 'True'].shape

(62, 18)

In [63]:
for index, row in genus_df.iterrows():         
    if index % 50 == 0:
        print(index, end=' ')
        
    if row['checked'] == 'True' or row['checked'] == True:
        continue
        
    time.sleep(0.3)   
    
    if pd.notna(row['pbdb_taxon_id']):
        url =  PBDB_TAXA_ID +  str(row['pbdb_taxon_id'])
    else:
        url =  PBDB_TAXA_NAME +  str(row['genus_taxon_name'])
        
    
    response = requests.get(url)
    fetch_pbdb_data(url, genus_df, 'genus')


0 50 100 150 200 250 300 350 400 450 500 550 600 650 700 750 800 850 900 950 1000 1050 1100 

In [64]:
genus_df[genus_df['checked'] == True].shape

(62, 18)

In [65]:
log_df(genus_df)

(1120, 18)


,genus_taxon_name,pbdb_taxon_rank,pbdb_taxon_name,pbdb_taxon_id,checked,genus_taxon_id,phylum_taxon_id,phylum_taxon_name,kingdom_taxon_id,kingdom_taxon_name,family_taxon_id,family_taxon_name,order_taxon_id,order_taxon_name,class_taxon_id,class_taxon_name,unranked clade_taxon_id,unranked clade_taxon_name
0,Euuvigerina,genus,Euuvigerina,1408,True,1408,288974,Foraminifera,212476,Rhizaria,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Foraminifera,phylum,Foraminifera,288974,True,288974,NaN,NaN,212476,Rhizaria,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Pleurostomellidae,family,Pleurostomellidae,155922,True,155922,288974,Foraminifera,212476,Rhizaria,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Ostracoda,class,Ostracoda,22826,True,22826,18891,Arthropoda,325038,Animalia,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Textulariia,subclass,Textulariia,434530,True,434530,288974,Foraminifera,212476,Rhizaria,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [67]:
genus_df.to_csv(genus_higher_file, index=False)

# Find changed taxa

In [86]:
def get_common_columns(new_file, old_file):
    df1 = pd.read_csv(new_file, dtype=str) 
    df2 = pd.read_csv(old_file, dtype=str)

    return list(set(df1.columns).intersection(set(df2.columns)))

def create_compare_df(df1, df2):
    merge_df = df1.merge(df2, how='outer',  indicator=True)
    changed_taxa = merge_df[merge_df['_merge'] != 'both'].copy()
    changed_taxa['pbdb_name_count'] = changed_taxa.groupby('pbdb_taxon_name')['pbdb_taxon_name'].transform('count')

    changed_taxa.loc[changed_taxa['_merge'] == 'left_only', 'version'] = 'updated'
    changed_taxa.loc[changed_taxa['_merge'] == 'right_only', 'version'] = 'old'

    return changed_taxa.sort_values(['pbdb_taxon_name','version'])


### genera

In [94]:
new_file =  genus_higher_file
old_file = old_taxa_list_file

same_cols = get_common_columns(new_file, old_file)

df1 = pd.read_csv(new_file , usecols =same_cols , dtype=str)
df1 = df1.drop_duplicates()

log_df(df1)


(1120, 15)


,genus_taxon_name,pbdb_taxon_rank,pbdb_taxon_name,pbdb_taxon_id,genus_taxon_id,phylum_taxon_id,phylum_taxon_name,kingdom_taxon_id,kingdom_taxon_name,family_taxon_id,family_taxon_name,order_taxon_id,order_taxon_name,class_taxon_id,class_taxon_name
0,Euuvigerina,genus,Euuvigerina,1408,1408,288974,Foraminifera,212476,Rhizaria,NaN,NaN,NaN,NaN,NaN,NaN
1,Foraminifera,phylum,Foraminifera,288974,288974,NaN,NaN,212476,Rhizaria,NaN,NaN,NaN,NaN,NaN,NaN
2,Pleurostomellidae,family,Pleurostomellidae,155922,155922,288974,Foraminifera,212476,Rhizaria,NaN,NaN,NaN,NaN,NaN,NaN
3,Ostracoda,class,Ostracoda,22826,22826,18891,Arthropoda,325038,Animalia,NaN,NaN,NaN,NaN,NaN,NaN
4,Textulariia,subclass,Textulariia,434530,434530,288974,Foraminifera,212476,Rhizaria,NaN,NaN,NaN,NaN,NaN,NaN


In [95]:

df2 = pd.read_csv(old_file , usecols =same_cols, dtype=str)

df2.loc[df2['pbdb_taxon_rank'] == 'species', 'pbdb_taxon_name'] = df2['genus_taxon_name']
df2.loc[df2['pbdb_taxon_rank'] == 'species', 'pbdb_taxon_id'] = df2['genus_taxon_id']
df2.loc[df2['pbdb_taxon_rank'] == 'species', 'pbdb_taxon_rank'] = 'genus'

df2 = df2.drop_duplicates().dropna(how='all')
log_df(df2)

(1120, 15)


,pbdb_taxon_id,pbdb_taxon_name,pbdb_taxon_rank,genus_taxon_id,genus_taxon_name,family_taxon_id,family_taxon_name,order_taxon_id,order_taxon_name,class_taxon_id,class_taxon_name,phylum_taxon_id,phylum_taxon_name,kingdom_taxon_id,kingdom_taxon_name
0,1408,Euuvigerina,genus,1408,Euuvigerina,NaN,NaN,NaN,NaN,NaN,NaN,288974,Foraminifera,212476,Rhizaria
2,288974,Foraminifera,phylum,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,288974,Foraminifera,212476,Rhizaria
3,155922,Pleurostomellidae,family,NaN,NaN,155922,Pleurostomellidae,NaN,NaN,NaN,NaN,288974,Foraminifera,212476,Rhizaria
4,22826,Ostracoda,class,NaN,NaN,NaN,NaN,NaN,NaN,22826,Ostracoda,18891,Arthropoda,325038,Animalia
5,434530,Textulariia,subclass,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,288974,Foraminifera,212476,Rhizaria


In [96]:
compare_df = create_compare_df(df1, df2)

# compare_df = compare_df[compare_df['taxa_count'] != 1]

cols = [
    'pbdb_taxon_id','pbdb_taxon_name','pbdb_taxon_rank',
    'genus_taxon_name',  'genus_taxon_id',
    'family_taxon_name', 'family_taxon_id',
    'order_taxon_name','order_taxon_id',
    'class_taxon_name', 'class_taxon_id',
    'phylum_taxon_name',  'phylum_taxon_id', 
    'kingdom_taxon_name','kingdom_taxon_id',
    'version', 'pbdb_name_count'
]

compare_df = compare_df.reindex(columns=cols)

log_df(compare_df)

(222, 17)


,pbdb_taxon_id,pbdb_taxon_name,pbdb_taxon_rank,genus_taxon_name,genus_taxon_id,family_taxon_name,family_taxon_id,order_taxon_name,order_taxon_id,class_taxon_name,class_taxon_id,phylum_taxon_name,phylum_taxon_id,kingdom_taxon_name,kingdom_taxon_id,version,pbdb_name_count
1142,141161,Acritarcha,unranked clade,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,old,2
559,141161,Acritarcha,unranked clade,Acritarcha,141161,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,updated,2
1165,54588,Alnus,genus,Alnus,54588,Betulaceae,54589,Fagales,54902,Magnoliopsida,55217,Spermatophyta,55350,Plantae,54311,old,2
728,54588,Alnus,genus,Alnus,54588,Betulaceae,54589,Fagales,54902,Angiospermae,452804,Spermatophyta,55350,Plantae,54311,updated,2
1166,434782,Anthemis,genus,Anthemis,434782,Asteraceae,183083,Asterales,54943,Magnoliopsida,55217,Spermatophyta,55350,Plantae,54311,old,2


In [97]:
compare_df.to_csv(OUTPUT_DIR/'taxa'/'draft'/'LIMS'/'compare_genus_and_higher_2023-02-09.csv', index=False)


### species

In [121]:
new_file = species_file
old_file = old_taxa_list_file

same_cols = get_common_columns(new_file, old_file)

df1 = pd.read_csv(new_file , usecols =same_cols , dtype=str)
df1 = df1.drop_duplicates()
df1 = df1.dropna(how='all')

log_df(df1)

(482, 17)


,pbdb_taxon_id,pbdb_taxon_name,pbdb_taxon_rank,species_taxon_id,species_taxon_name,genus_taxon_id,genus_taxon_name,family_taxon_id,family_taxon_name,phylum_taxon_id,phylum_taxon_name,kingdom_taxon_id,kingdom_taxon_name,order_taxon_id,order_taxon_name,class_taxon_id,class_taxon_name
2,422278,Candeina nitida,species,422278,Candeina nitida,1053,Candeina,422277,Candeinidae,288974,Foraminifera,212476,Rhizaria,NaN,NaN,NaN,NaN
3,402661,Dentoglobigerina altispira,species,402661,Dentoglobigerina altispira,1264,Dentoglobigerina,82191,Globigerinidae,288974,Foraminifera,212476,Rhizaria,NaN,NaN,NaN,NaN
4,422289,Dentoglobigerina baroemoenensis,species,422289,Dentoglobigerina baroemoenensis,1264,Dentoglobigerina,82191,Globigerinidae,288974,Foraminifera,212476,Rhizaria,NaN,NaN,NaN,NaN
5,113301,Globigerina bulloides,species,113301,Globigerina bulloides,1498,Globigerina,82191,Globigerinidae,288974,Foraminifera,212476,Rhizaria,NaN,NaN,NaN,NaN
6,388387,Globigerina falconensis,species,388387,Globigerina falconensis,1498,Globigerina,82191,Globigerinidae,288974,Foraminifera,212476,Rhizaria,NaN,NaN,NaN,NaN


In [122]:

df2 = pd.read_csv(old_file , usecols =same_cols, dtype=str)
df2 = df2[df2['pbdb_taxon_rank'] == 'species']

df2 = df2.drop_duplicates().dropna(how='all')
log_df(df2)

(482, 17)


,pbdb_taxon_id,pbdb_taxon_name,pbdb_taxon_rank,species_taxon_id,species_taxon_name,genus_taxon_id,genus_taxon_name,family_taxon_id,family_taxon_name,order_taxon_id,order_taxon_name,class_taxon_id,class_taxon_name,phylum_taxon_id,phylum_taxon_name,kingdom_taxon_id,kingdom_taxon_name
9,422278,Candeina nitida,species,422278,Candeina nitida,1053,Candeina,422277,Candeinidae,NaN,NaN,NaN,NaN,288974,Foraminifera,212476,Rhizaria
10,402661,Dentoglobigerina altispira,species,402661,Dentoglobigerina altispira,1264,Dentoglobigerina,82191,Globigerinidae,NaN,NaN,NaN,NaN,288974,Foraminifera,212476,Rhizaria
11,422289,Dentoglobigerina baroemoenensis,species,422289,Dentoglobigerina baroemoenensis,1264,Dentoglobigerina,82191,Globigerinidae,NaN,NaN,NaN,NaN,288974,Foraminifera,212476,Rhizaria
12,113301,Globigerina bulloides,species,113301,Globigerina bulloides,1498,Globigerina,82191,Globigerinidae,NaN,NaN,NaN,NaN,288974,Foraminifera,212476,Rhizaria
13,388387,Globigerina falconensis,species,388387,Globigerina falconensis,1498,Globigerina,82191,Globigerinidae,NaN,NaN,NaN,NaN,288974,Foraminifera,212476,Rhizaria


In [123]:
compare_df = create_compare_df(df1, df2)

cols = [
    'pbdb_taxon_id', 'pbdb_taxon_name', 'pbdb_taxon_rank',
    'species_taxon_name', 'species_taxon_id',
    'genus_taxon_name',  'genus_taxon_id',
    'family_taxon_name', 'family_taxon_id',
    'order_taxon_name','order_taxon_id',
    'class_taxon_name', 'class_taxon_id',
    'phylum_taxon_name',  'phylum_taxon_id', 
    'kingdom_taxon_name','kingdom_taxon_id',
    'version'
]

compare_df = compare_df.reindex(columns=cols)


In [124]:
compare_df.to_csv(OUTPUT_DIR/'taxa'/'draft'/'LIMS'/'compare_species_2023-02-09.csv', index=False)


# merge basic taxa file taxa file with pdbdb data

Add pbdb info from  previous genus file with pbdb datat to file without pdbd data

In [68]:
genus_df = pd.read_csv(genus_higher_file, dtype=str)
del genus_df['checked']
log_df(genus_df)

(1120, 17)


,genus_taxon_name,pbdb_taxon_rank,pbdb_taxon_name,pbdb_taxon_id,genus_taxon_id,phylum_taxon_id,phylum_taxon_name,kingdom_taxon_id,kingdom_taxon_name,family_taxon_id,family_taxon_name,order_taxon_id,order_taxon_name,class_taxon_id,class_taxon_name,unranked clade_taxon_id,unranked clade_taxon_name
0,Euuvigerina,genus,Euuvigerina,1408,1408,288974,Foraminifera,212476,Rhizaria,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Foraminifera,phylum,Foraminifera,288974,288974,NaN,NaN,212476,Rhizaria,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Pleurostomellidae,family,Pleurostomellidae,155922,155922,288974,Foraminifera,212476,Rhizaria,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Ostracoda,class,Ostracoda,22826,22826,18891,Arthropoda,325038,Animalia,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Textulariia,subclass,Textulariia,434530,434530,288974,Foraminifera,212476,Rhizaria,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [56]:
p = OUTPUT_DIR/'taxa'/'LIMS'/f'genera_pbdb_{date}_4.csv'
prev_genus_df = pd.read_csv(p, dtype=str)


In [57]:

merge_df = genus_df.merge(prev_genus_df,  how='left')
log_df(merge_df)

(1120, 18)


,genus_taxon_name,pbdb_taxon_rank,pbdb_taxon_name,pbdb_taxon_id,checked,genus_taxon_id,phylum_taxon_id,phylum_taxon_name,kingdom_taxon_id,kingdom_taxon_name,family_taxon_id,family_taxon_name,order_taxon_id,order_taxon_name,class_taxon_id,class_taxon_name,unranked clade_taxon_id,unranked clade_taxon_name
0,Euuvigerina,genus,Euuvigerina,1408,True,1408,288974,Foraminifera,212476,Rhizaria,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,phylum,Foraminifera,288974,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,family,Pleurostomellidae,155922,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,class,Ostracoda,22826,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,subclass,Textulariia,434530,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [58]:
merge_df[merge_df['pbdb_taxon_id'].isna()]

,genus_taxon_name,pbdb_taxon_rank,pbdb_taxon_name,pbdb_taxon_id,checked,genus_taxon_id,phylum_taxon_id,phylum_taxon_name,kingdom_taxon_id,kingdom_taxon_name,family_taxon_id,family_taxon_name,order_taxon_id,order_taxon_name,class_taxon_id,class_taxon_name,unranked clade_taxon_id,unranked clade_taxon_name


In [60]:
merge_df.to_csv(genus_higher_file, index=False)

# Not done: fix species

pbdb returns genus species as species name. we want species name to only be species

In [53]:
species_df = pd.read_csv(species_file, dtype=str)
log_df(species_df)

(3267, 21)


,genus species name,pbdb_taxon_id,checked,pbdb_taxon_name,pbdb_taxon_rank,species_taxon_id,species_taxon_name,genus_taxon_id,genus_taxon_name,family_taxon_id,...,phylum_taxon_id,phylum_taxon_name,kingdom_taxon_id,kingdom_taxon_name,order_taxon_id,order_taxon_name,class_taxon_id,class_taxon_name,unranked clade_taxon_id,unranked clade_taxon_name
0,Euuvigerina miozea,NaN,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Euuvigerina rodleyi,NaN,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Candeina nitida,422278,True,Candeina nitida,species,422278,Candeina nitida,1053,Candeina,422277,...,288974,Foraminifera,212476,Rhizaria,NaN,NaN,NaN,NaN,NaN,NaN
3,Dentoglobigerina altispira,402661,True,Dentoglobigerina altispira,species,402661,Dentoglobigerina altispira,1264,Dentoglobigerina,82191,...,288974,Foraminifera,212476,Rhizaria,NaN,NaN,NaN,NaN,NaN,NaN
4,Dentoglobigerina baroemoenensis,422289,True,Dentoglobigerina baroemoenensis,species,422289,Dentoglobigerina baroemoenensis,1264,Dentoglobigerina,82191,...,288974,Foraminifera,212476,Rhizaria,NaN,NaN,NaN,NaN,NaN,NaN


In [63]:
for index, row in species_df.iterrows():
    if(pd.notna(row['species_taxon_name']) and pd.notna(row['pbdb_taxon_id'])):
        species_df.loc[index, 'species_taxon_name']  = row['species_taxon_name'].replace(row['genus_taxon_name'], '').strip()

In [64]:
log_df(species_df)

(3267, 22)


,genus species name,pbdb_taxon_id,checked,pbdb_taxon_name,pbdb_taxon_rank,species_taxon_id,species_taxon_name,genus_taxon_id,genus_taxon_name,family_taxon_id,...,phylum_taxon_name,kingdom_taxon_id,kingdom_taxon_name,order_taxon_id,order_taxon_name,class_taxon_id,class_taxon_name,unranked clade_taxon_id,unranked clade_taxon_name,tmp
0,Euuvigerina miozea,NaN,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Euuvigerina rodleyi,NaN,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Candeina nitida,422278,True,Candeina nitida,species,422278,nitida,1053,Candeina,422277,...,Foraminifera,212476,Rhizaria,NaN,NaN,NaN,NaN,NaN,NaN,Candeina nitida
3,Dentoglobigerina altispira,402661,True,Dentoglobigerina altispira,species,402661,altispira,1264,Dentoglobigerina,82191,...,Foraminifera,212476,Rhizaria,NaN,NaN,NaN,NaN,NaN,NaN,Dentoglobigerina altispira
4,Dentoglobigerina baroemoenensis,422289,True,Dentoglobigerina baroemoenensis,species,422289,baroemoenensis,1264,Dentoglobigerina,82191,...,Foraminifera,212476,Rhizaria,NaN,NaN,NaN,NaN,NaN,NaN,Dentoglobigerina baroemoenensis


In [65]:
species_df['tmp'] = species_df[['genus_taxon_name', 'species_taxon_name']].stack().groupby(level=0).agg(' '.join)
log_df(species_df)

(3267, 22)


,genus species name,pbdb_taxon_id,checked,pbdb_taxon_name,pbdb_taxon_rank,species_taxon_id,species_taxon_name,genus_taxon_id,genus_taxon_name,family_taxon_id,...,phylum_taxon_name,kingdom_taxon_id,kingdom_taxon_name,order_taxon_id,order_taxon_name,class_taxon_id,class_taxon_name,unranked clade_taxon_id,unranked clade_taxon_name,tmp
0,Euuvigerina miozea,NaN,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Euuvigerina rodleyi,NaN,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Candeina nitida,422278,True,Candeina nitida,species,422278,nitida,1053,Candeina,422277,...,Foraminifera,212476,Rhizaria,NaN,NaN,NaN,NaN,NaN,NaN,Candeina nitida
3,Dentoglobigerina altispira,402661,True,Dentoglobigerina altispira,species,402661,altispira,1264,Dentoglobigerina,82191,...,Foraminifera,212476,Rhizaria,NaN,NaN,NaN,NaN,NaN,NaN,Dentoglobigerina altispira
4,Dentoglobigerina baroemoenensis,422289,True,Dentoglobigerina baroemoenensis,species,422289,baroemoenensis,1264,Dentoglobigerina,82191,...,Foraminifera,212476,Rhizaria,NaN,NaN,NaN,NaN,NaN,NaN,Dentoglobigerina baroemoenensis


In [68]:
foo_df = species_df[species_df['tmp'] != species_df['genus species name']]
foo_df = foo_df[foo_df['pbdb_taxon_id'].notna()]
foo_df

,genus species name,pbdb_taxon_id,checked,pbdb_taxon_name,pbdb_taxon_rank,species_taxon_id,species_taxon_name,genus_taxon_id,genus_taxon_name,family_taxon_id,...,phylum_taxon_name,kingdom_taxon_id,kingdom_taxon_name,order_taxon_id,order_taxon_name,class_taxon_id,class_taxon_name,unranked clade_taxon_id,unranked clade_taxon_name,tmp
7,Globigerina rubescens,422320,True,Globigerina rubescens,species,422320,Globigerina rubescens,1529,Globoturborotalita,82191,...,Foraminifera,212476,Rhizaria,NaN,NaN,NaN,NaN,NaN,NaN,Globoturborotalita Globigerina rubescens
19,Globigerinoides sacculifer,388389,True,Globigerinoides sacculifer,species,388389,Globigerinoides sacculifer,422361,Trilobatus,82191,...,Foraminifera,212476,Rhizaria,NaN,NaN,NaN,NaN,NaN,NaN,Trilobatus Globigerinoides sacculifer
23,Globorotalia crassaformis,422430,True,Globorotalia (Truncorotalia) crassaformis,species,422430,(Truncorotalia) crassaformis,1521,Globorotalia,82192,...,Foraminifera,212476,Rhizaria,NaN,NaN,NaN,NaN,NaN,NaN,Globorotalia (Truncorotalia) crassaformis
25,Globorotalia humerosa,402487,True,Globorotalia humerosa,species,402487,Globorotalia humerosa,1917,Neogloboquadrina,82192,...,Foraminifera,212476,Rhizaria,NaN,NaN,NaN,NaN,NaN,NaN,Neogloboquadrina Globorotalia humerosa
26,Globorotalia limbata,422439,True,Globorotalia (Menardella) limbata,species,422439,(Menardella) limbata,1521,Globorotalia,82192,...,Foraminifera,212476,Rhizaria,NaN,NaN,NaN,NaN,NaN,NaN,Globorotalia (Menardella) limbata
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2973,Thyrsocyrtis tensa,152673,True,Thyrsocyrtis (Pentalacorys) tensa,species,152673,(Pentalacorys) tensa,685,Thyrsocyrtis,87065,...,Radiolaria,212476,Rhizaria,402,Nassellaria,NaN,NaN,NaN,NaN,Thyrsocyrtis (Pentalacorys) tensa
2975,Thyrsocyrtis triacantha,152671,True,Thyrsocyrtis (Pentalacorys) triacantha,species,152671,(Pentalacorys) triacantha,685,Thyrsocyrtis,87065,...,Radiolaria,212476,Rhizaria,402,Nassellaria,NaN,NaN,NaN,NaN,Thyrsocyrtis (Pentalacorys) triacantha
2990,Distephanus boliviensis,435386,True,Distephanus boliviensis,species,435386,Distephanus boliviensis,434215,Stephanocha,69592,...,NaN,NaN,NaN,69591,Phytomastigophora,69590,Chrysophyceae,28595,Life,Stephanocha Distephanus boliviensis
2992,Distephanus speculum,98653,True,Distephanus speculum,species,98653,Distephanus speculum,434215,Stephanocha,69592,...,NaN,NaN,NaN,69591,Phytomastigophora,69590,Chrysophyceae,28595,Life,Stephanocha Distephanus speculum


decided not to fix things since species name somtimes don't start with geni=us name